In [ ]:
#Get meK_looms.tar.gz and meKMeans_data_samp_params.csv



samps = pd.read_csv('meKMeans_data_samp_params.csv')
samps.head()

In [ ]:
#For each loom in looms from ./hvg_objs

#Get tech param by keyword (datas) in loom paths

#For 



#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, NMI


#sklearn.metrics.adjusted_rand_score()
#sklearn.metrics.adjusted_mutual_info_score()

### **meK-Means**


In [ ]:
#Correct K + 5 + 10 higher 

### **Leiden & K-Means**


In [ ]:
#Make the U,S, U+S, Concat matrices, res 0.8,1,1.5

#K-Means just correct K